# <font color="orange">Check for updates</font>

**Current version : 2024.01.15**

In [ ]:
import ipywidgets as widgets
from IPython.utils import capture
with capture.capture_output() as cap: 
    !pip install -q -U requests
import requests

button = widgets.Button(description="Download now", button_style='success')
output = widgets.Output()

def on_button_clicked(b, url, version):
  with output:
    print('Downloading...')
    !wget -nv -O /notebooks/sd_webui_paperspace_{version}.ipynb {url}
    print(f'sd_webui_paperspace_{version}.ipynb downloaded in the root directory')

currentVersion = '2024.01.15'
updateURL = 'https://raw.githubusercontent.com/ffxvs/sd-webui-complete-setup/main/update.json'
response = requests.get(updateURL)

if response.status_code == 200:
    res = response.json()
    for data in res['data']:
        if data['id'] == 'paperspace':
            for variant in data['variants']:
                if variant['type'] == 'sd':
                    print('Current version : ' + currentVersion)
                    print('Latest version  : ' + variant['version'])
                    if (variant['version'] > currentVersion):
                        print('\nThere is new version')
                        button.on_click(lambda b: on_button_clicked(b, variant['url'], variant['version']))
                        display(button, output)
                        print('\nChangelog :')
                        print(variant['changelog'])
                    else:
                        print('\nThis is the latest version')                
else:
    print('Failed to check for updates') 
    print('Response code :', response.status_code)

# <font color="orange">1. Requirements</font>

### 1.1. Function and Data
**REQUIRED EVERY TIME YOU RELOAD**

In [ ]:
# Function and Data (REQUIRED)

import os, re
import ipywidgets as widgets
from IPython.utils import capture
from IPython.display import clear_output

########################### GLOBAL PATHS AND FUNCTION ###########################

boolean = [False, True]

# Paths
root = '/notebooks'
sharedStorage = root + '/shared-storage'
sharedModelsPath = sharedStorage + '/models'
sharedEmbeddingsPath = sharedStorage + '/embeddings'
sharedLoraPath = sharedStorage + '/lora'
sharedUpscalerPath = sharedStorage + '/esrgan'
sharedHypernetworkspPath = sharedStorage + '/hypernetworks'
sharedVaePath = sharedStorage + '/vae'
sharedOutputsPath = sharedStorage + '/outputs'
sharedConfigPath = sharedStorage + '/config'
webui = root + '/stable-diffusion-webui'
modulesPath = webui + '/modules'
outputsPath = webui + '/outputs'
extensionsPath = webui + '/extensions'
controlNetModelsPath = extensionsPath + "/sd-webui-controlnet/models"
embeddingsPath = webui + "/embeddings"
othersEmbeddingsPath = embeddingsPath + '/others'
modelsPath = webui + "/models/Stable-diffusion"
loraPath = webui + "/models/Lora"
hypernetworksPath = webui + "/models/hypernetworks"
upscalerPath = webui + "/models/ESRGAN"
vaePath = webui + "/models/VAE"


# Create symlink
def symlink(source, destination):
    if os.path.exists(source) and not os.path.islink(destination):
        !rm -r -f {destination}
        !ln -s {source} {destination}

# Complete message
def inf(msg, style, width): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=width));display(inf)

# Model Mapper
def modelMapper(name, version, url):
    return {
        'name': name,
        'version': version,
        'url': url
    }

# Selected Models
def selectedModels(models):
    modelList = []
    for model in models:
        isSelected = eval(model['id'])
        if isSelected != 'Select version...':
            for variant in model['variants']:
                if variant['version'] == isSelected:
                    modelList.append(modelMapper(
                        model['name'],
                        variant['version'],
                        variant['url']
                    ))
    return modelList

# Aria2c
def downloader(url, path, overwrite=False):
    args = '--download-result=hide --console-log-level=error -c -x 16 -s 16 -k 1M '
    if overwrite: args += '--allow-overwrite'
    formattedURL = '"' + url + '"'
    if bool(re.search('\/[\w\.-]+\.\w+$', url)):
        filename = url.split('/')[-1]
        !aria2c {args} {formattedURL} -d {path} -o {filename}
    else:
        !aria2c {args} {formattedURL} -d {path}

# Git Clone
def silentClone(command, path, overwrite=False):
    directory = command.split('/')[-1]
    if os.path.exists(path + '/' + directory):
        if overwrite:
            !rm -r {path}/{directory}
            !git clone -q {command} {path}/{directory}
    else:
        !git clone -q {command} {path}/{directory}

# WGet
def silentGet(command):
    !wget -nv {command}

# Resources downloader
def defaultResources(resourceList, resourcePath):
    for resource in resourceList:
        print('\n' + resource['name'] + '...')
        downloader(resource['url'], resourcePath)

def selectedResources(resourceList, resourcePath):
    for resource in resourceList:
        if eval(resource['id']):
            print('\n' + resource['name'] + '...')
            for url in resource['url']:
                downloader(url, resourcePath)
                print('')
                
def otherResources(resourceList, resourcePath):
    for resource in resourceList:
        print('\n' + resource)
        downloader(resource, resourcePath)

# Remove git from directory
def removeGit(directory):
    for (folder, subfolders, files) in os.walk(directory):
        if subfolders:
            for subfolder in subfolders:
                if subfolder == '.git':
                    pathFolder = os.path.join(folder, subfolder)
                    !rm -r {pathFolder}

        for file in files:
            if file == '.gitattributes' or file == 'readme.md' or file == 'README.md':
                pathFile = os.path.join(folder, file)
                !rm {pathFile}


################################ DATA OBJECT ################################
################################ CONTROLNET ################################

controlNetURLs = {
    'canny': [
        'https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny_fp16.safetensors',
        'https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_canny_fp16.yaml',
    ],
    'depth': [
        'https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors',
        'https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1p_sd15_depth_fp16.yaml',
    ],
    'inpaint': [
        'https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors',
        'https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_inpaint_fp16.yaml',
    ],
    'ip2p': [
        'https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors',
        'https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11e_sd15_ip2p_fp16.yaml',
    ],
    'ipadapter': [
        'https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_sd15.pth',
        'https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_sd15_plus.pth',
    ],
    'lineart': [
        'https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart_fp16.safetensors',
        'https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_lineart_fp16.yaml',
    ],
    'lineart_anime': [
        'https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors',
        'https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15s2_lineart_anime_fp16.yaml',
    ],
    'mlsd': [
        'https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors',
        'https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_mlsd_fp16.yaml',
    ],
    'normal': [
        'https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors',
        'https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_normalbae_fp16.yaml',
    ],
    'openpose': [
        'https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose_fp16.safetensors',
        'https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_openpose_fp16.yaml',
    ],
    'scribble': [
        'https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble_fp16.safetensors',
        'https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_scribble_fp16.yaml',
    ],
    'seg': [
        'https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg_fp16.safetensors',
        'https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_seg_fp16.yaml',
    ],
    'shuffle': [
        'https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors',
        'https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11e_sd15_shuffle_fp16.yaml',
    ],
    'softedge': [
        'https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge_fp16.safetensors',
        'https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_softedge_fp16.yaml',
    ],
    'tile': [
        'https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors',
        'https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1e_sd15_tile_fp16.yaml',
    ],
}

t2iAdapterURLs = [
    'https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_style_sd14v1.pth',
    'https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_sketch_sd14v1.pth',
    'https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_seg_sd14v1.pth',
    'https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_openpose_sd14v1.pth',
    'https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_keypose_sd14v1.pth',
    'https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_depth_sd14v1.pth',
    'https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_color_sd14v1.pth',
    'https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_canny_sd14v1.pth',
    'https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_canny_sd15v2.pth',
    'https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_depth_sd15v2.pth',
    'https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_sketch_sd15v2.pth',
    'https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_zoedepth_sd15v1.pth',
]

################################ EXTENSIONS ################################

defaultExtensions = [
    {
        'name': 'Aspect Ratio Helper',
        'url': 'https://github.com/thomasasfk/sd-webui-aspect-ratio-helper',
    },
    {
        'name': 'Canvas Zoom',
        'url': 'https://github.com/richrobber2/canvas-zoom',
    },
    {
        'name': 'Cleaner',
        'url': 'https://github.com/novitalabs/sd-webui-cleaner',
    },
    {
        'name': 'Controlnet',
        'url': 'https://github.com/Mikubill/sd-webui-controlnet',
    },
    {
        'name': 'SD Delete Button',
        'url': 'https://github.com/reforget-id/sd_delete_button',
    },
    {
        'name': 'SD Model Downloader',
        'url': 'https://github.com/Iyashinouta/sd-model-downloader',
    },
    {
        'name': 'State',
        'url': 'https://github.com/ilian6806/stable-diffusion-webui-state',
    },
    {
        'name': 'Ultimate SD Upscale',
        'url': 'https://github.com/Coyote-A/ultimate-upscale-for-automatic1111',
    }
]

extensionList = [
    {
        'id': 'adetailer',
        'name': 'Adetailer (After Detailer)',
        'url': 'https://github.com/Bing-su/adetailer',
    },
    {
        'id': 'animateDiff',
        'name': 'AnimateDiff',
        'url': 'https://github.com/continue-revolution/sd-webui-animatediff',
    },
    {
        'id': 'depthLib',
        'name': 'Depth Map Library',
        'url': 'https://github.com/wywywywy/sd-webui-depth-lib',
    },
    {
        'id': 'huggingFace',
        'name': 'Hugging Face',
        'url': 'https://github.com/camenduru/stable-diffusion-webui-huggingface',
    },
    {
        'id': 'infiniteImg',
        'name': 'Infinite Image Browsing',
        'url': 'https://github.com/zanllp/sd-webui-infinite-image-browsing',
    },
    {
        'id': 'inpaintAny',
        'name': 'Inpaint Anything',
        'url': 'https://github.com/Uminosachi/sd-webui-inpaint-anything',
    },
    {
        'id': 'latentCouple',
        'name': 'Latent Couple',
        'url': 'https://github.com/aria1th/stable-diffusion-webui-two-shot',
    },
    {
        'id': 'miniPaint',
        'name': 'Mini Paint',
        'url': 'https://github.com/0Tick/a1111-mini-paint',
    },
    {
        'id': 'npw',
        'name': 'Negative Prompt Weight',
        'url': 'https://github.com/muerrilla/stable-diffusion-NPW',
    },
    {
        'id': 'openOutpaint',
        'name': 'openOutpaint',
        'url': 'https://github.com/zero01101/openOutpaint-webUI-extension',
    },
    {
        'id': 'photopea',
        'name': 'Photopea',
        'url': 'https://github.com/yankooliveira/sd-webui-photopea-embed',
    },
    {
        'id': 'promptHistory',
        'name': 'Prompt History',
        'url': 'https://github.com/namkazt/sd-webui-prompt-history',
    },
    {
        'id': 'rembg',
        'name': 'Remove Background',
        'url': 'https://github.com/AUTOMATIC1111/stable-diffusion-webui-rembg',
    },
    {
        'id': 'regionalPrompter',
        'name': 'Regional Prompter',
        'url': 'https://github.com/hako-mikan/sd-webui-regional-prompter',
    },
    {
        'id': 'sag',
        'name': 'Self Attention Guidance',
        'url': 'https://github.com/ashen-sensored/sd_webui_SAG',
    },
    {
        'id': 'tagComplete',
        'name': 'Tag Autocomplete',
        'url': 'https://github.com/DominikDoom/a1111-sd-webui-tagcomplete',
    },
    {
        'id': 'tiledDiffusion',
        'name': 'Tiled Diffusion & VAE',
        'url': 'https://github.com/pkuliyi2015/multidiffusion-upscaler-for-automatic1111',
    },
]


#################################### MODELS #####################################
############################# ANIME / CARTOON /3D ###############################

animeModels = [
    {
        'id': 'anything',
        'name': 'Anything',
        'variants': [
            {
                'version': 'Ink',
                'url': 'https://huggingface.co/X779/Anything_ink/resolve/main/Anything-ink.safetensors'
            },
        ]
    },
    {
        'id': 'babes',
        'name': 'Babes',
        'variants': [
            {
                'version': 'v3',
                'url': 'https://huggingface.co/alexds9/Babes_3.0/resolve/main/Babes_3.0_fp16.safetensors?download=true'
            },
        ]
    },
    {
        'id': 'cetusMix',
        'name': 'Cetus-Mix',
        'variants': [
            {
                'version': 'WhaleFall2',
                'url': 'https://civitai.com/api/download/models/105924?type=Model&format=SafeTensor&size=pruned&fp=fp16'
            },
        ]
    },
    {
        'id': 'counterfeit',
        'name': 'Counterfeit',
        'variants': [
            {
                'version': 'v3',
                'url': 'https://huggingface.co/gsdf/Counterfeit-V3.0/resolve/main/Counterfeit-V3.0_fix_fp16.safetensors'
            },
        ]
    },
    {
        'id': 'meinaMix',
        'name': 'MeinaMix',
        'variants': [
            {
                'version': 'v11',
                'url': 'https://huggingface.co/asiryan/meinamix/resolve/main/meinamix_meinaV11.safetensors'
            },
            {
                'version': 'v11-inpainting',
                'url': 'https://huggingface.co/asiryan/meinamix/resolve/main/meinamix_v11-inpainting.safetensors'
            },
        ]
    },
    {
        'id': 'revAnimated',
        'name': 'ReV Animated',
        'variants': [
            {
                'version': 'v1.2.2',
                'url': 'https://huggingface.co/s6yx/ReV_Animated/resolve/main/rev_1.2.2/rev_1.2.2-fp16.safetensors?download=true'
            },
        ]
    },
    {
        'id': 'toonYou',
        'name': 'ToonYou',
        'variants': [
            {
                'version': 'beta 6',
                'url': 'https://huggingface.co/SavageSally/toonyou_beta6/resolve/main/toonyou_beta6.safetensors'
            },
        ]
    }
]

############################### GENERAL PURPOSE #################################

generalModels = [
    {
        'id': 'deliberate',
        'name': 'Deliberate',
        'variants': [
            {
                'version': 'v5',
                'url': 'https://huggingface.co/XpucT/Deliberate/resolve/main/Deliberate_v5.safetensors'
            },
            {
                'version': 'v5-inpainting',
                'url': 'https://huggingface.co/XpucT/Deliberate/resolve/main/Deliberate_v5-inpainting.safetensors'
            },
        ]
    },
    {
        'id': 'dreamShaper',
        'name': 'DreamShaper',
        'variants': [
            {
                'version': 'v8',
                'url': 'https://huggingface.co/Lykon/DreamShaper/resolve/main/DreamShaper_8_pruned.safetensors'
            },
            {
                'version': 'v8-inpainting',
                'url': 'https://huggingface.co/Lykon/DreamShaper/resolve/main/DreamShaper_8_INPAINTING.inpainting.safetensors'
            },
        ]
    },
    {
        'id': 'experience',
        'name': 'Experience',
        'variants': [
            {
                'version': 'Exp v10',
                'url': 'https://huggingface.co/Schisim/Experience/resolve/main/Experience_v10-FP16.safetensors'
            },
            {
                'version': 'Real Exp v3',
                'url': 'https://huggingface.co/Schisim/Experience/resolve/main/Realistic_Experience_v3-FP16.safetensors'
            },
        ]
    },
    {
        'id': 'ned',
        'name': 'NeverEnding Dream (NED)',
        'variants': [
            {
                'version': 'v1.22_vae',
                'url': 'https://huggingface.co/Lykon/NeverEnding-Dream/resolve/main/NeverEndingDream_1.22_BakedVae_fp16.safetensors'
            },
            {
                'version': 'v1.22-inpainting',
                'url': 'https://huggingface.co/fp16-guy/NeverEnding-Dream_fp16_cleaned/resolve/main/neverendingDreamNED_v122BakedVae_inp_fp16.safetensors'
            },
        ]
    }, 
    {
        'id': 'protogen',
        'name': 'Protogen',
        'variants': [
            {
                'version': 'x2.2 (Anime)',
                'url': 'https://huggingface.co/darkstorm2150/Protogen_v2.2_Official_Release/resolve/main/Protogen_V2.2-pruned-fp16.safetensors'
            },
            {
                'version': 'x3.4 (Photorealism)',
                'url': 'https://huggingface.co/darkstorm2150/Protogen_x3.4_Official_Release/resolve/main/ProtoGen_X3.4-pruned-fp16.safetensors'
            },
            {
                'version': 'x5.3 (Photorealism)',
                'url': 'https://huggingface.co/darkstorm2150/Protogen_x5.3_Official_Release/resolve/main/ProtoGen_X5.3-pruned-fp16.safetensors'
            },
            {
                'version': 'x5.8 (Sci-fi + Anime)',
                'url': 'https://huggingface.co/darkstorm2150/Protogen_x5.8_Official_Release/resolve/main/ProtoGen_X5.8-pruned-fp16.safetensors'
            },
        ]
    },
]

################################## REALISTIC ####################################

realisticModels = [
    {
        'id': 'absoluteReality',
        'name': 'Absolute Reality',
        'variants': [
            {
                'version': 'v1.8.1',
                'url': 'https://huggingface.co/Lykon/AbsoluteReality/resolve/main/AbsoluteReality_1.8.1_pruned.safetensors'
            },
            {
                'version': 'v1.8.1-inpainting',
                'url': 'https://huggingface.co/Lykon/AbsoluteReality/resolve/main/AbsoluteReality_1.8.1_INPAINTING.inpainting.safetensors'
            },
        ]
    },
    {
        'id': 'analogMadness',
        'name': 'Analog Madness',
        'variants': [
            {
                'version': 'v7',
                'url': 'https://civitai.com/api/download/models/261539?type=Model&format=SafeTensor&size=pruned&fp=fp16'
            },
            {
                'version': 'v7-inpainting',
                'url': 'https://civitai.com/api/download/models/269280?type=Model&format=SafeTensor&size=full&fp=fp16'
            },
        ]
    },
    {
        'id': 'cyberRealistic',
        'name': 'CyberRealistic',
        'variants': [
            {
                'version': 'v4.1',
                'url': 'https://huggingface.co/cyberdelia/CyberRealistic/resolve/main/CyberRealistic_V4.1_FP16.safetensors'
            },
            {
                'version': 'v4.1-inpainting',
                'url': 'https://civitai.com/api/download/models/256955?type=Model&format=SafeTensor&size=pruned&fp=fp16'
            },
        ]
    },
    {
        'id': 'epicPhotogasm',
        'name': 'epiCPhotoGasm',
        'variants': [
            {
                'version': 'Last Unicorn',
                'url': 'https://huggingface.co/jzli/epiCPhotoGasm-last-unicorn/resolve/main/epicphotogasm_lastUnicorn.safetensors'
            },
            {
                'version': 'Z',
                'url': 'https://civitai.com/api/download/models/201259?type=Model&format=SafeTensor&size=pruned&fp=fp16'
            },
            {
                'version': 'Z-inpainting',
                'url': 'https://civitai.com/api/download/models/201346?type=Model&format=SafeTensor&size=pruned&fp=fp16'
            },
        ]
    },
    {
        'id': 'epicRealism',
        'name': 'epiCRealism',
        'variants': [
            {
                'version': 'Natural Sin',
                'url': 'https://huggingface.co/Magamanny/epiCRealism/resolve/main/naturalSin/epicrealism_naturalSin.safetensors'
            },
            {
                'version': 'Pure Evo v5',
                'url': 'https://huggingface.co/Magamanny/epiCRealism/resolve/main/pureEvolution/epicrealism_pureEvolutionV5.safetensors'
            },
            {
                'version': 'Pure Evo v5-inpainting',
                'url': 'https://huggingface.co/Lola12/epicrealism_pureEvolutionV5-inpainting/resolve/main/epicrealism_pureEvolutionV5-inpainting.safetensors'
            },
        ]
    },
    {
        'id': 'realVision',
        'name': 'Realistic Vision',
        'variants': [
            {
                'version': 'v6_b1_vae',
                'url': 'https://civitai.com/api/download/models/245598?type=Model&format=SafeTensor&size=pruned&fp=fp16'
            },
            {
                'version': 'v6_b1_vae-inpainting',
                'url': 'https://civitai.com/api/download/models/245627?type=Model&format=SafeTensor&size=pruned&fp=fp16'
            },
        ]
    },
    {
        'id': 'reliberate',
        'name': 'Reliberate',
        'variants': [
            {
                'version': 'v3',
                'url': 'https://huggingface.co/XpucT/Reliberate/resolve/main/Reliberate_v3.safetensors'
            },
            {
                'version': 'v3-inpainting',
                'url': 'https://huggingface.co/XpucT/Reliberate/resolve/main/Reliberate_v3-inpainting.safetensors'
            },
        ]
    },
]

#################################### WOMAN #####################################

womanModels = [
    {
        'id': 'bra',
        'name': 'BRA (Beautiful Realistic Asians)',
        'variants': [
            {
                'version': 'v7',
                'url': 'https://huggingface.co/apikachu/beautifulRealistic_v7/resolve/main/beautifulRealistic_v7.safetensors'
            },
        ]
    },
    {
        'id': 'bem',
        'name': 'BracingEvoMix',
        'variants': [
          {
            'version': 'v2',
            'url': 'https://huggingface.co/sazyou-roukaku/BracingEvoMix/resolve/main/BracingEvoMix_v2.safetensors'
          },
        ]
    },
    {
        'id': 'chilledRemix',
        'name': 'Chilled Remix',
        'variants': [
          {
            'version': 'Remix v2',
            'url': 'https://huggingface.co/sazyou-roukaku/chilled_remix/resolve/main/chilled_remix_v2.safetensors'
          },
             {
            'version': 'Reversemix v2',
            'url': 'https://huggingface.co/sazyou-roukaku/chilled_remix/resolve/main/chilled_reversemix_v2.safetensors'
          },
        ]
    },
    {
        'id': 'henmixReal',
        'name': 'Henmix Real',
        'variants': [
          {
            'version': 'v4',
            'url': 'https://huggingface.co/BBlou/henmix/resolve/main/henmixReal_v40.safetensors'
          },
        ]
    },
    {
        'id': 'majicMixReal',
        'name': 'majicMIX Realistic',
        'variants': [
          {
            'version': 'v7',
            'url': 'https://huggingface.co/marcy1111/majicmixRealistic_v7/resolve/main/majicmixRealistic_v7.safetensors'
          },
            {
            'version': 'v7-inpainting',
            'url': 'https://civitai.com/api/download/models/176425?type=Model&format=SafeTensor&size=pruned&fp=fp16'
          },
        ]
    },
]


########################################################################################################
############################################## LORA ####################################################

defaultLoraList = [
    {
        'name': 'Add More Details - Detail Enhancer/Tweaker',
        'url': 'https://huggingface.co/ffxvs/lora-effects/resolve/main/add_more_details.safetensors',
    },
    {
        'name': 'Detail Tweaker',
        'url': 'https://huggingface.co/ffxvs/lora-effects/resolve/main/detail_tweaker.safetensors',
    },
    {
        'name': 'Epi Noise Offset',
        'url': 'https://huggingface.co/ffxvs/lora-effects/resolve/main/epiNoiseoffset_v2.safetensors',
    },
    {
        'name': 'LowRA',
        'url': 'https://huggingface.co/ffxvs/lora-effects/resolve/main/lowRA.safetensors',
    },
]

loraList = [
    {
        'id': 'bgDetails',
        'name': 'Intricate Background & Background Blur',
        'url': ['https://huggingface.co/ffxvs/lora-effects/resolve/main/background_details.safetensors'],
    },
    {
        'id': 'dopSlider',
        'name': 'Depth of Field Slider',
        'url': ['https://huggingface.co/ffxvs/lora-effects/resolve/main/depth_of_field_slider.safetensors'],
    },
    {
        'id': 'dollLikeness',
        'name': 'Doll Likeness Series',
        'url': [
            'https://huggingface.co/Kanbara/doll-likeness-series/resolve/main/japaneseDollLikeness_v15.safetensors',
            'https://huggingface.co/Kanbara/doll-likeness-series/resolve/main/koreanDollLikeness_v20.safetensors',
            'https://huggingface.co/Kanbara/doll-likeness-series/resolve/main/taiwanDollLikeness_v20.safetensors',
            'https://huggingface.co/Kanbara/doll-likeness-series/resolve/main/ThaiDollLikeness_v10.safetensors',
            'https://huggingface.co/Kanbara/doll-likeness-series/resolve/main/chinaDollLikeness_v10.safetensors'
        ],
    },
    {
        'id': 'lcmLora',
        'name': 'LCM Lora SD v1.5',
        'url': ['https://huggingface.co/ffxvs/lora-effects/resolve/main/lcm_lora_sd15.safetensors'],
    }
]


################################### EMBEDDINGS (TEXTUAL INVERSION) ####################################

defaultEmbeddingList = [
    {
        'name': 'Negative prompts pack',
        'url': 'https://huggingface.co/ffxvs/negative-prompts-pack',
    },
    {
        'name': 'Embeddings Collection',
        'url': 'https://huggingface.co/ffxvs/embeddings-collection',
    },
]

embeddingList = [
    {
        'id': 'scgEmotion',
        'name': 'SCG Emotions Pack',
        'url': 'https://huggingface.co/ffxvs/scg-emotion-pack',
    },
    {
        'id': 'scgToolkit',
        'name': 'SCG Embedding Toolkit',
        'url': 'https://huggingface.co/ffxvs/scg-toolkit',
    },
    {
        'id': 'ageSlider',
        'name': 'Zovya Age Slider',
        'url': 'https://huggingface.co/ffxvs/zovya-age-slider',
    },
    {
        'id': 'genderSlider',
        'name': 'Zovya Gender Slider',
        'url': 'https://huggingface.co/ffxvs/zovya-gender-slider',
    },
]


############################################## UPSCALER ##############################################

defaultUpscalerList = [
    {
        'name': 'UltraSharp',
        'url': 'https://huggingface.co/ffxvs/upscaler/resolve/main/UltraSharp_4x.pth',
    },
]

upscalerList = [
    {
        'id': 'bsrgan',
        'name': 'BSRGAN',
        'url': ['https://huggingface.co/ffxvs/upscaler/resolve/main/BSRGAN_4x.pth'],
    },
    {
        'id': 'lsdirPlus',
        'name': 'LSDIRPlus',
        'url': ['https://huggingface.co/ffxvs/upscaler/resolve/main/LSDIRplus_4x.pth'],
    },
    {
        'id': 'nomos8ksc',
        'name': 'Nomos8kSC',
        'url': ['https://huggingface.co/ffxvs/upscaler/resolve/main/Nomos8kSC_4x.pth'],
    },
    {
        'id': 'superscale',
        'name': 'NMKD Superscale',
        'url': ['https://huggingface.co/ffxvs/upscaler/resolve/main/NMKD_Superscale_4x.pth'],
    },
    {
        'id': 'remacri',
        'name': 'Remacri',
        'url': ['https://huggingface.co/ffxvs/upscaler/resolve/main/Remacri_4x.pth'],
    },
    {
        'id': 'ultraMix',
        'name': 'UltraMix',
        'url': [
            'https://huggingface.co/ffxvs/upscaler/resolve/main/UltraMix_Balanced_4x.pth',
            'https://huggingface.co/ffxvs/upscaler/resolve/main/UltraMix_Restore_4x.pth',
            'https://huggingface.co/ffxvs/upscaler/resolve/main/UltraMix_Smooth_4x.pth'
        ],
    },
    {
        'id': 'valar',
        'name': 'Valar',
        'url': ['https://huggingface.co/ffxvs/upscaler/resolve/main/Valar_4x.pth'],
    },
]


############################################## VAE ##############################################

defaultVaeList = [
    {
        'name': 'vae-ft-mse-840000',
        'url': 'https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors',
    },
]

vaeList = [
    {
        'id': 'blessed2',
        'name': 'Blessed2',
        'desc': 'iCoMix',
        'url': ['https://huggingface.co/ffxvs/vae-collection/resolve/main/blessed2.vae.pt'],
    },
    {
        'id': 'clearVAE',
        'name': 'ClearVAE',
        'desc': 'Anime models',
        'url': ['https://huggingface.co/ffxvs/vae-collection/resolve/main/clearVAE.safetensors'],
    },
    {
        'id': 'kl_f8_anime2',
        'name': 'kl-f8-anime2',
        'desc': 'CetusMix, Fantasy World, PastelMix, Waifu, Yozora',
        'url': ['https://huggingface.co/ffxvs/vae-collection/resolve/main/kl-f8-anime2.ckpt'],
    },
    {
        'id': 'novelAI',
        'name': 'NAI/NovelAI',
        'desc': 'Anything v3, Counterfeit, OrangeMix',
        'url': ['https://huggingface.co/ffxvs/vae-collection/resolve/main/novelAI.vae.pt'],
    },
]

inf('\u2714 Completed','success', '30px')

### 1.2. Dependencies
**REQUIRED EVERY RUNTIME START**

In [ ]:
%cd {root}
print('---------------------------------------')
print('⏳ Installing dependencies...')
print('It will take a long time...')

with capture.capture_output() as cap:
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
    os.environ['PYTHONWARNINGS'] = 'ignore'

    !apt-get -y -q update
    !apt-get -y -q install aria2 git-lfs
    !pip install -q -U torch torchvision torchaudio torchtext torchdata --index-url https://download.pytorch.org/whl/cu118
    !pip install -q -U ngrok triton insightface

print('\n')
inf('\u2714 Completed','success', '30px')

### 1.3. Shared Storage
**After running this cell, put your `config.json` and `ui-config.json` to `shared-storage/config`** if you have them  
**Only needs to be run once**

In [ ]:
# Make shared storage symlink
%cd /
symlink('/storage', sharedStorage)

# Make necessary folders in shared storage if not exists
%cd {sharedStorage}
!mkdir -p {sharedModelsPath}
!mkdir -p {sharedEmbeddingsPath}
!mkdir -p {sharedLoraPath}
!mkdir -p {sharedUpscalerPath}
!mkdir -p {sharedHypernetworkspPath}
!mkdir -p {sharedVaePath}
!mkdir -p {sharedOutputsPath}
!mkdir -p {sharedConfigPath}

%cd {root}
print('\n')
inf('\u2714 Completed','success', '30px')

# <font color="orange">2. Setup Webui</font>

### 2.1. Webui

In [ ]:
print('⏳ Installing Stable Diffusion Webui...')
silentClone('-b v1.7.0 --depth 1 https://github.com/AUTOMATIC1111/stable-diffusion-webui', root)
%cd {webui}

# Make symlinks from shared storage
symlink(sharedModelsPath, modelsPath)
symlink(sharedLoraPath, loraPath)
symlink(sharedEmbeddingsPath, othersEmbeddingsPath)
symlink(sharedUpscalerPath, upscalerPath)
symlink(sharedHypernetworkspPath, hypernetworksPath)
symlink(sharedVaePath, vaePath)
symlink(sharedOutputsPath, outputsPath)
symlink(f'{sharedConfigPath}/config.json', f'{webui}/config.json')
symlink(f'{sharedConfigPath}/ui-config.json', f'{webui}/ui-config.json')

# From ThelastBen
!mkdir -p {webui}/cache
os.environ['TRANSFORMERS_CACHE'] = f'{webui}/cache'
os.environ['TORCH_HOME'] = f'{webui}/cache'

downloader('https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/modules/extras.py', modulesPath, True)
downloader('https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/modules/sd_models.py', modulesPath, True)
downloader('https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/modules/shared_options.py', modulesPath, True)

!sed -i 's@shared.opts.data\["sd_model_checkpoint"] = checkpoint_info.title@shared.opts.data\["sd_model_checkpoint"] = checkpoint_info.title;model.half()@' {modulesPath}/sd_models.py
!sed -i "s@map_location='cpu'@map_location='cuda'@" {modulesPath}/extras.py
!sed -i 's@\["sd_model_checkpoint"\]@\["sd_model_checkpoint", "sd_vae", "CLIP_stop_at_last_layers"\]@g' {modulesPath}/shared_options.py

print('\n')
inf('\u2714 Completed','success', '30px')

### 2.2. Extensions

#### Pre-Installed Extensions

**This is just for information. No need to run it.**  

* [Aspect Ratio Helper](https://github.com/thomasasfk/sd-webui-aspect-ratio-helper)
* [Canvas Zoom](https://github.com/richrobber2/canvas-zoom)
* [Cleaner](https://github.com/novitalabs/sd-webui-cleaner)
* [Controlnet](https://github.com/Mikubill/sd-webui-controlnet)
* [SD Delete Button](https://github.com/reforget-id/sd_delete_button)
* [SD Model Downloader](https://github.com/Iyashinouta/sd-model-downloader)
* [State](https://github.com/ilian6806/stable-diffusion-webui-state)
* [Ultimate SD Upscale](https://github.com/Coyote-A/ultimate-upscale-for-automatic1111)

#### Extension List

In [ ]:
# ControlNet - https://github.com/Mikubill/sd-webui-controlnet
allControlNet = boolean[0] # Download all controlNet models

controlNet = {  # Select models to download 
                # Default models are : depth, inpaint, ip-adapter, openpose, tile
    'canny': boolean[0],
    'depth': boolean[1],
    'inpaint': boolean[1],
    'ip2p': boolean[0],
    'ipadapter': boolean[1],
    'lineart': boolean[0],
    'lineart_anime': boolean[0],
    'mlsd': boolean[0],
    'normal': boolean[0],
    'openpose': boolean[1],
    'scribble': boolean[0],
    'seg': boolean[0],
    'shuffle': boolean[0],
    'softedge': boolean[0],
    'tile': boolean[1],
}

# T2I-Adapter - https://github.com/TencentARC/T2I-Adapter
t2iAdapter = boolean[0]

# Adetailer (After Detailer) - https://github.com/Bing-su/adetailer
adetailer = boolean[1]

# AnimateDiff - https://github.com/continue-revolution/sd-webui-animatediff
animateDiff = boolean[0]

# Depth Map Library - https://github.com/wywywywy/sd-webui-depth-lib
depthLib = boolean[0]

# Hugging Face - https://github.com/camenduru/stable-diffusion-webui-huggingface
huggingFace = boolean[0]

# Infinite Image Browsing - https://github.com/zanllp/sd-webui-infinite-image-browsing
infiniteImg = boolean[1]

# Inpaint Anything - https://github.com/Uminosachi/sd-webui-inpaint-anything
inpaintAny = boolean[0]

# Latent Couple - https://github.com/aria1th/stable-diffusion-webui-two-shot
latentCouple = boolean[0]

# Mini Paint - https://github.com/0Tick/a1111-mini-paint
miniPaint = boolean[1]

# Negative Prompt Weight - https://github.com/muerrilla/stable-diffusion-NPW
npw = boolean[0]

# openOutpaint - https://github.com/zero01101/openOutpaint-webUI-extension
openOutpaint = boolean[0]

# Photopea - https://github.com/yankooliveira/sd-webui-photopea-embed
photopea = boolean[1]

# Prompt History - https://github.com/namkazt/sd-webui-prompt-history
promptHistory = boolean[0]

# Regional Prompter - https://github.com/hako-mikan/sd-webui-regional-prompter
regionalPrompter = boolean[0]

# Remove Background - https://github.com/AUTOMATIC1111/stable-diffusion-webui-rembg
rembg = boolean[0]

# Self Attention Guidance - https://github.com/ashen-sensored/sd_webui_SAG
sag = boolean[0]

# Tag Autocomplete - https://github.com/DominikDoom/a1111-sd-webui-tagcomplete
tagComplete = boolean[1]

# Tiled Diffusion & VAE - https://github.com/pkuliyi2015/multidiffusion-upscaler-for-automatic1111
tiledDiffusion = boolean[1]


################################################################################################################


print("⏳ Installing pre-installed extensions...")
for extension in defaultExtensions:
    print(extension['name'] + '...')
    silentClone(extension['url'], extensionsPath)

print("\n⏳ Installing selected extensions...")
for extension in extensionList:
    if eval(extension['id']):
        print(extension['name'] + '...')
        silentClone(extension['url'], extensionsPath)

if allControlNet:
    print("\n⏳ Download all acontrolNet models...")
    for model in controlNetURLs:
        print('\n' + model + '...')
        downloader(controlNetURLs[model][0], controlNetModelsPath)
        downloader(controlNetURLs[model][1], controlNetModelsPath)
else:
    print("\n⏳ Download selected controlNet models...")
    for model in controlNet:
        if controlNet[model]:
            print('\n' + model + '...')
            downloader(controlNetURLs[model][0], controlNetModelsPath)
            downloader(controlNetURLs[model][1], controlNetModelsPath)        

if t2iAdapter:
    print("\n⏳ Download T2I-Adapter models...")
    for model in t2iAdapterURLs:
        print('\n' + model)
        downloader(model, controlNetModelsPath)

print('\n')        

addNetLoraPath = extensionsPath + '/sd-webui-additional-networks/models/lora'
if os.path.exists(addNetLoraPath):
    !mkdir -p {loraPath}
    !rm -r {addNetLoraPath}
    !ln -s {sharedLoraPath} {addNetLoraPath}

%cd {webui}
print('\n')
inf('\u2714 Completed','success', '30px')

#### Install from URL

In [ ]:
# Install extensions from URL (separate by comma)
otherExtensions = []

if otherExtensions:
    print("⏳ Installing other extensions...")  
    for extension in otherExtensions:
        name = extension.split('/')[-1]
        print(name + '...')
        silentClone(extension, extensionsPath)

print('\n')        
%cd {webui}
print('\n')
inf('\u2714 Completed','success', '30px')

# <font color="orange">3. Models</font>
**Choose models you want to download**

### Anime / Cartoon / 3D

In [ ]:
# Anything - https://civitai.com/models/9409
anythingVersions = ['Select version...', 'Ink']
anything = anythingVersions[0]

# Babes - https://civitai.com/models/2220
babesVersions = ['Select version...', 'v3']
babes = babesVersions[0]

# Cetus-Mix - https://civitai.com/models/6755
cetusMixVersions = ['Select version...', 'WhaleFall2']
cetusMix = cetusMixVersions[0]

# Counterfeit - https://civitai.com/models/4468
counterfeitVersions = ['Select version...', 'v3']
counterfeit = counterfeitVersions[0]

# MeinaMix - https://civitai.com/models/7240
meinaMixVersions = ['Select version...', 'v11', 'v11-inpainting']
meinaMix = meinaMixVersions[0]

# ReV Animated - https://civitai.com/models/7371
revAnimatedVersions = ['Select version...', 'v1.2.2']
revAnimated = revAnimatedVersions[0]

# ToonYou - https://civitai.com/models/30240
toonYouVersions = ['Select version...', 'beta 6']
toonYou = toonYouVersions[0]


########################################################################################


print('⏳ Downloading selected models...')
for model in selectedModels(animeModels):
    print('\n* ' + model['name'] + ' | ' + model['version'])
    downloader(model['url'], modelsPath)

print('\n') 
inf('\u2714 Completed','success', '30px')

### General Purpose

In [ ]:
# Deliberate - https://huggingface.co/XpucT/Deliberate
deliberateVersions = ['Select version...', 'v5', 'v5-inpainting']
deliberate = deliberateVersions[0]

# DreamShaper - https://civitai.com/models/4384
dreamShaperVersions = ['Select version...', 'v8', 'v8-inpainting']
dreamShaper = dreamShaperVersions[0]

# Experience - https://civitai.com/models/5952/experience
experienceVersions = ['Select version...', 'Exp v10', 'Real Exp v3']
experience = experienceVersions[0]

# NeverEnding Dream (NED) - https://civitai.com/models/10028
nedVersions = ['Select version...', 'v1.22_vae', 'v1.22-inpainting']
ned = nedVersions[0]

# Protogen - https://civitai.com/user/darkstorm2150
protogenVersions = ['Select version...', 
    'x2.2 (Anime)', 
    'x3.4 (Photorealism)', 
    'x5.3 (Photorealism)', 
    'x5.8 (Sci-fi + Anime)'
]
protogen = protogenVersions[0]


########################################################################################


print('⏳ Downloading selected models...')
for model in selectedModels(generalModels):
    print('\n* ' + model['name'] + ' | ' + model['version'])
    downloader(model['url'], modelsPath)

print('\n') 
inf('\u2714 Completed','success', '30px')

### Realistic

In [ ]:
# Absolute Reality - https://civitai.com/models/81458
absoluteRealityVersions = ['Select version...', 'v1.8.1', 'v1.8.1-inpainting']
absoluteReality = absoluteRealityVersions[0]

# Analog Madness - https://civitai.com/models/8030
analogMadnessVersions = ['Select version...', 'v7', 'v7-inpainting']
analogMadness = analogMadnessVersions[0]

# CyberRealistic - https://civitai.com/models/15003
cyberRealisticVersions = ['Select version...', 'v4.1', 'v4.1-inpainting']
cyberRealistic = cyberRealisticVersions[0]

# epiCPhotoGasm - https://civitai.com/models/132632
epicPhotogasmVersions = ['Select version...', 'Last Unicorn', 'Z', 'Z-inpainting']
epicPhotogasm = epiCPhotoGasmVersions[0]

# epiCRealism - https://civitai.com/models/25694
epicRealismVersions = ['Select version...', 'Natural Sin', 'Pure Evo v5', 'Pure Evo v5-inpainting']
epicRealism = epicRealismVersions[0]

# Realistic Vision - https://civitai.com/models/4201
realVisionVersions = ['Select version...', 'v6_b1_vae', 'v6_b1-inpainting']
realVision = realVisionVersions[0]

# Reliberate - https://huggingface.co/XpucT/Reliberate
reliberateVersions = ['Select version...', 'v3', 'v3-inpainting']
reliberate = reliberateVersions[0]


########################################################################################


print('⏳ Downloading selected models...')
for model in selectedModels(realisticModels):
    print('\n* ' + model['name'] + ' | ' + model['version'])
    downloader(model['url'], modelsPath)

print('\n') 
inf('\u2714 Completed','success', '30px')

### Woman

In [ ]:
# BRA (Beautiful Realistic Asians) - https://civitai.com/models/25494
braVersions = ['Select version...', 'v7']
bra = braVersions[1]

# BracingEvoMix - https://huggingface.co/sazyou-roukaku/BracingEvoMix
bemVersions = ['Select version...', 'v2']
bem = bemVersions[0]

# Chilled Remix - https://huggingface.co/sazyou-roukaku/chilled_remix
chilledRemixVersions = ['Select version...', 'Remix v2', 'Reversemix v2']
chilledRemix = chilledRemixVersions[0]

# Henmix Real - https://civitai.com/models/20282
henmixRealVersions = ['Select version...', 'v4']
henmixReal = henmixRealVersions[0]

# majicMIX Realistic - https://civitai.com/models/43331
majicMixRealVersions = ['Select version...', 'v7', 'v7-inpainting']
majicMixReal = majicMixRealVersions[0]


########################################################################################


print('⏳ Downloading selected models...')
for model in selectedModels(womanModels):
    print('\n* ' + model['name'] + ' | ' + model['version'])
    downloader(model['url'], modelsPath)

print('\n') 
inf('\u2714 Completed','success', '30px')

### Install from URL

In [ ]:
# Install models from URL 
otherModels = []


########################################################################################


if otherModels:
    print('⏳ Downloading other models...')
    for model in otherModels:
        print('\n* ' + model)
        downloader(model, modelsPath)
        
print('\n')
%cd {webui}
print('\n')
inf('\u2714 Completed','success', '30px')

# <font color="orange">4. Resources</font>

### Pre-Installed Resources

**This is just for information. No need to run it.**  

**Pre-installed LoRA :**
* [Add More Details - Detail Enhancer/Tweaker](https://civitai.com/models/82098)
* [Detail Tweaker](https://civitai.com/models/58390)
* [Epi Noise Offset](https://civitai.com/models/13941/epinoiseoffset)
* [LowRA](https://civitai.com/models/48139/lowra)

**Pre-installed Embeddings :**
* [Embeddings Collection](https://huggingface.co/ffxvs/embeddings-collection)
* [Negative Prompts Pack](https://huggingface.co/ffxvs/negative-prompts-pack)

**Pre-installed Upscaler :**
* [UltraSharp](https://openmodeldb.info/models/4x-UltraSharp)

**Pre-installed VAE :**
* [vae-ft-mse-840000](https://huggingface.co/stabilityai/sd-vae-ft-mse-original)

### LoRA, Embedding, Hypernetwork, Upscaler and VAE

In [ ]:
####################################### LORA ##########################################

# Depth of Field Slider - https://civitai.com/models/135380
dopSlider = boolean[0]

# Doll Likeness Series - https://huggingface.co/Kanbara/doll-likeness-series
dollLikeness = boolean[0]

# Intricate Background & Background Blur - https://civitai.com/models/105732
bgDetails = boolean[0]

# LCM Lora SD v1.5 - https://huggingface.co/latent-consistency/lcm-lora-sdv1-5
lcmLora = boolean[0]

# Install LoRA from URL
otherLora = []

############################ EMBEDDINGS (TEXTUAL INVERSION) ############################

# SCG Emotions Pack - https://civitai.com/models/8860
scgEmotion = boolean[0]

# SCG Embedding Toolkit - https://civitai.com/models/8551
scgToolkit = boolean[0]

# Zovya Age Slider - https://civitai.com/models/65214
ageSlider = boolean[0]

# Zovya Gender Slider - https://civitai.com/models/89709
genderSlider = boolean[0]

# Install embeddings from URL
otherEmbeddings = []

###################################### UPSCALER ########################################

# BSRGAN - https://openmodeldb.info/models/4x-BSRGAN
bsrgan = boolean[0]

# LSDIRPlus - https://openmodeldb.info/models/4x-LSDIRplus
lsdirPlus = boolean[0]

# NMKD Superscale - https://openmodeldb.info/models/4x-NMKD-Superscale
superscale = boolean[0]

# Nomos8kSC - https://openmodeldb.info/models/4x-Nomos8kSC
nomos8ksc = boolean[0]

# Remacri - https://openmodeldb.info/models/4x-Remacri
remacri = boolean[0]

# UltraMix - https://upscale.wiki/w/index.php?title=Model_Database&oldid=1571
ultraMix = boolean[0]

# Valar - https://openmodeldb.info/models/4x-Valar
valar = boolean[0]

# Install upscaler from URL
otherUpscaler = []

######################################## VAE ############################################

# Blessed2 - https://civitai.com/models/118561?modelVersionId=142467
blessed2 = boolean[0]

# ClearVAE - https://civitai.com/models/22354
clearVAE = boolean[0]

# kl-f8-anime2 - https://civitai.com/models/23906
kl_f8_anime2 = boolean[0]

# NAI/NovelAI - https://huggingface.co/WarriorMama777/OrangeMixs
novelAI = boolean[0]

# Install VAE from URL
otherVAE = []

#################################### HYPERNETWORKS ######################################

# Install Hypernetworks from URL
otherHypernetworks = []

###################################################################################################


# Make necessary directories if not exists
!mkdir -p {loraPath}
!mkdir -p {embeddingsPath}
!mkdir -p {upscalerPath}
!mkdir -p {hypernetworksPath}
%cd {webui}

# LoRA
def downloadLora():
    print('⏳ Downloading pre-installed LoRA...')
    defaultResources(defaultLoraList, loraPath)
    print('\n\n⏳ Downloading selected LoRA...')
    selectedResources(loraList, loraPath)    
    if otherLora:
        print('\n\n⏳ Downloading other LoRA...')
        otherResources(otherLora, loraPath)

# Embeddings
def downloadEmbeddings():
    print('\n\n⏳ Downloading pre-installed embeddings...')
    for embedding in defaultEmbeddingList:
        print(embedding['name'] + '...')
        silentClone(embedding['url'], embeddingsPath, True)

    print('\n\n⏳ Downloading selected embeddings...')
    for embedding in embeddingList:
        if eval(embedding['id']):
            print(embedding['name'] + '...')
            silentClone(embeddings['url'], embeddingsPath, True)
            
    removeGit(embeddingsPath)
    if otherEmbeddings:
        print('\n\n⏳ Downloading other embeddings...')
        otherResources(otherEmbeddings, othersEmbeddingsPath)

# Hypernetworks
def downloadHypernetworks():
    if otherHypernetworks:
        print('\n\n⏳ Downloading other hypernetworks...')
        otherResources(otherHypernetworks, hypernetworksPath)

# Upscaler
def downloadUpscaler():
    print('\n\n⏳ Downloading pre-installed upscaler...')
    defaultResources(defaultUpscalerList, upscalerPath)
    print('\n\n⏳ Downloading selected upscaler...')
    selectedResources(upscalerList, upscalerPath)
    if otherUpscaler:
        print('\n\n⏳ Downloading other upscaler...')
        otherResources(otherUpscaler, upscalerPath)

# VAE 
def downloadVAE():
    print('\n\n⏳ Downloading pre-installed VAE...')
    defaultResources(defaultVaeList, vaePath)
    print('\n\n⏳ Downloading selected VAE...')
    selectedResources(vaeList, vaePath)
    if otherVAE:
        print('\n\n⏳ Downloading other VAE...')
        otherResources(otherVAE, vaePath)

# Download LoRA & others
def downloadAllResources():
    downloadLora()
    downloadEmbeddings()
    downloadHypernetworks()
    downloadUpscaler()
    downloadVAE()

downloadAllResources()

print('\n')
%cd {webui}
print('\n')
inf('\u2714 Completed','success', '30px')

# <font color="orange">5. Launch Webui</font>

Wait until `Model loaded in **.*s ...` appears.  
Then, click Gradio link `https://xxxxx.gradio.live` or Ngrok link `https://xxxxx.ngrok-free.app` to open WebUI.  
Always watch the output log to track image generation progress or errors. 

In [ ]:
boolean = [False, True]

# Dark theme
darkTheme = boolean[1]

# Optimizations
# For "opt-sdp-attention" and "opt-sdp-no-mem-attention" set directly from webui settings
xformers = boolean[1]
medvram = boolean[0]

# Ngrok (Optional)
ngrokToken = ''
ngrokDomain = ''

# CORS (Optional) 
# separate with commas
cors = 'https://huchenlei.github.io'


################################################################################################################

import json 
%cd {webui}
print('⏳ Preparing...')
args = '--disable-safe-unpickle --enable-insecure-extension-access --gradio-queue --api --listen'

if darkTheme:
    args += ' --theme dark'
if xformers:
    args += ' --xformers'
if medvram:
    args += ' --medvram'
if ngrokToken:
    args += ' --ngrok ' + ngrokToken
    if ngrokDomain:
        ngrokOptions = '\'{"hostname":"' + ngrokDomain + '", "request_header.add":"ngrok-skip-browser-warning:skip"}\''
        args += ' --ngrok-options ' + ngrokOptions
else:
    args += ' --share'
if cors:
    args += ' --cors-allow-origins ' + cors

with capture.capture_output() as cap:
    if xformers:
        !pip install -q -U xformers --index-url https://download.pytorch.org/whl/cu118
    !python launch.py --exit
    !pip install -q pillow==9.5.0

print('Launching Webui...')
!python launch.py {args}

# <font color="orange">6. Zip Outputs</font>
After the output folder is zipped, you can download it from file browser in the left panel.  
It's in `stable-diffusion-webui` folder. Just right click to download.

In [ ]:
%cd {webui}
print('⏳ Compressing files into outputs.zip...')
!zip -r -q outputs.zip outputs
print('The file is ready in /stable-diffusion-webui/outputs.zip')

inf('\u2714 Completed','success', '30px')